In [1]:
from cgra import *
kernel_name = "transformer"
# Input data
kernel_clear_memory(kernel_name)
kernel_add_memory_region(kernel_name, 100, [10, 20, 30, 40, 50, 60,70,80,90,100,110,120,130,140,150,160,170,180,190,200])
kernel_add_memory_region(kernel_name, 1000, [100,104,108,112, 4,1, 1, 2, 3, 4, 5, 6,7,8,9,10,11,12,13,14,15,16,17,18,19])
# Set load and write addrs
load_addrs = [1000, 1016, 1020, 1000+(4+2+4*3)*4]
store_addrs = [0, 16, 32, 48]
run(kernel_name, pr=["ROUT","INST"], load_addrs=load_addrs, store_addrs=store_addrs)

Instr =  0 ( 0 )
[ 100,    4,    1,    0]    [
[ 104,    0,    0,    0]    [
[ 108,    0,    0,    0]    [
[ 112,    0,    0,    0]    [
-------
Instr =  1 ( 1 )
[ 100,    4,    0,    0]    [
[ 104,    0,    0,    0]    [
[ 108,    0,    0,    0]    [
[ 112,    0,    0,    0]    [
-------
Instr =  2 ( 2 )
[ 100,    4,   18,    0]    [
[ 104,    0,    0,    0]    [
[ 108,    0,    0,    0]    [
[ 112,    0,    0,    0]    [
-------
Instr =  3 ( 3 )
[   0,    0,    0,    0]    [
[   0,    0,    0,    0]    [
[   0,    0,    0,    0]    [
[   0,    0,    0,    0]    [
-------
Instr =  4 ( 4 )
[ 100,    0,    0,    0]    [
[ 104,    0,    0,    0]    [
[ 108,    0,    0,    0]    [
[ 112,    0,    0,    0]    [
-------
Instr =  5 ( 5 )
[ 100,   15,    0,    0]    [
[ 104,    0,    0,    0]    [
[ 108,    0,    0,    0]    [
[ 112,    0,    0,    0]    [
-------
Instr =  6 ( 6 )
[   1,    5,    9,   13]    [
[ 104,    0,    0,    0]    [
[ 108,    0,    0,    0]    [
[ 112,    0,    0,    0

In [1]:
from sat_to_csv import *
from kernels import *
from cgra import *
kernel_name = "transformer"
convert( kernel_name+"/out.sat", kernel_name+"/instructions.csv" )